In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from glob import glob

In [22]:
processed_data_path = "data/stocks/processed"
stocks_file_paths = glob(processed_data_path + "/*")
datetime_strings = [os.path.basename(path).split('_')[0] for path in stocks_file_paths]
for path in stocks_file_paths:
    if max(datetime_strings) in path:
        stocks_file_path = path
        print(f"Newest file: {stocks_file_path}")
        break

Newest file: data/stocks/processed/20220824T074737_xetra.csv


In [23]:
df_stocks = pd.read_csv(stocks_file_path)
df_stocks.set_index('ISIN', drop=True, inplace=True)
df_stocks.head(3)

,STOCK,CURRENCY
ISIN,,
DE0005545503,1+1 AG INH O.N.,EUR
DE0005118806,11 88 0 SOLUTIONS AG,EUR
DE000A2P4HL9,123FAHRSCHULE SE INH O.N.,EUR


In [76]:
df_stocks


,STOCK,CURRENCY
ISIN,,
DE0005545503,1+1 AG INH O.N.,EUR
DE0005118806,11 88 0 SOLUTIONS AG,EUR
DE000A2P4HL9,123FAHRSCHULE SE INH O.N.,EUR
DE000A0HL8N9,2G ENERGY AG,EUR
DE000A3H3L44,2INVEST AG NA O.N.,EUR
...,...,...
US9892071054,"ZEBRA TECH. A DL-,01",EUR
US98978V1035,"ZOETIS INC. CL.A DL -,01",EUR
US98980L1017,"ZOOM VIDEO COMM. A -,001",EUR


In [93]:
base_url = "https://www.finanzen.net"


In [92]:
stock_properties_for_all_stocks = []
for search_string in df_stocks.index[:20]:
    print(search_string)
    api_finanzendotnet = f"https://www.finanzen.net/suchergebnis.asp?strSuchString={search_string}"
    response = requests.get(api_finanzendotnet)
    soup = BeautifulSoup(response.text, 'html.parser')

    stock_properties = dict()
    stock_name = soup.find('h1', {'class': 'snapshot__headline'}).text
    stock_properties["stock_name"] = stock_name

    for instrument_id in soup.find('div', {'class': 'badge-bar'}).find_all('h2', {'class': 'badge background-color-de-black-haze pointer display-none-md margin-vertical-0.00'}):
        key, value = instrument_id.text.split(' ', 1)
        stock_properties[key] = value

    news_link = soup.find('a', {'class': 'display-none-md font-whitespace-nowrap-md'}).attrs['href']
    stock_properties["news_link"] = news_link
    stock_properties_for_all_stocks.append(stock_properties)


DE0005545503
DE0005118806
DE000A2P4HL9
DE000A0HL8N9
DE000A3H3L44
US88554D2053
US88579Y1010
DE0005167902
LU2380748603
GB00BMCLYF79
DE000A3E5C40
DE000A11QW68
DK0010244425


KeyboardInterrupt: 

In [86]:
print(stock_properties_for_all_stocks)

[{'stock_name': '1&1 Aktie', 'WKN': '554550', 'ISIN': 'DE0005545503', 'Symbol': '', 'news_link': '/news/1_und_1-news'}, {'stock_name': '11880 Solutions Aktie', 'WKN': '511880', 'ISIN': 'DE0005118806', 'Symbol': 'SLLNF', 'news_link': '/news/11880_solutions-news'}, {'stock_name': '123fahrschule Aktie', 'WKN': 'A2P4HL', 'ISIN': 'DE000A2P4HL9', 'Symbol': '', 'news_link': '/news/123fahrschule-news'}, {'stock_name': '2G Energy Aktie', 'WKN': 'A0HL8N', 'ISIN': 'DE000A0HL8N9', 'Symbol': 'ZGBEF', 'news_link': '/news/2g_energy-news'}, {'stock_name': '2invest Aktie', 'WKN': 'A3H3L4', 'ISIN': 'DE000A3H3L44', 'Symbol': 'FBSBF', 'news_link': '/analysen/2invest-analysen'}, {'stock_name': '3D Systems Aktie', 'WKN': '888346', 'ISIN': 'US88554D2053', 'Symbol': 'DDD', 'news_link': '/news/3d_systems-news'}, {'stock_name': '3M Aktie', 'WKN': '851745', 'ISIN': 'US88579Y1010', 'Symbol': 'MMM', 'news_link': '/news/3m-news'}, {'stock_name': '3U Aktie', 'WKN': '516790', 'ISIN': 'DE0005167902', 'Symbol': '', 'ne

In [88]:
df_finanzen = pd.DataFrame(stock_properties_for_all_stocks)


In [89]:
df_finanzen.to_csv("data/stocks/finanzen-dot-net-basics.csv", index=False)

In [90]:
df_finanzen

,stock_name,WKN,ISIN,Symbol,news_link
0,1&1 Aktie,554550,DE0005545503,,/news/1_und_1-news
1,11880 Solutions Aktie,511880,DE0005118806,SLLNF,/news/11880_solutions-news
2,123fahrschule Aktie,A2P4HL,DE000A2P4HL9,,/news/123fahrschule-news
3,2G Energy Aktie,A0HL8N,DE000A0HL8N9,ZGBEF,/news/2g_energy-news
4,2invest Aktie,A3H3L4,DE000A3H3L44,FBSBF,/analysen/2invest-analysen
5,3D Systems Aktie,888346,US88554D2053,DDD,/news/3d_systems-news
6,3M Aktie,851745,US88579Y1010,MMM,/news/3m-news
7,3U Aktie,516790,DE0005167902,,/news/3u-news
8,468 SPAC II Aktie,A3C81B,LU2380748603,,/news/468_spac_ii-news
9,4basebio Aktie,A3CWA4,GB00BMCLYF79,,/analysen/4basebio_2-analysen


In [96]:
url_news = base_url + "/news/1_und_1-news"


In [97]:
response = requests.get(url_news)
soup = BeautifulSoup(response.text, 'html.parser')


In [125]:
# extract first page

for news in soup.find_all('div', {'class': 'news news--item-with-media'}):
    date = news.find('time', {'class': 'news__date'}).text.encode('latin').decode()
    source = news.find('span', {'class': 'news__source'}).text.encode('latin').decode()
    kicker = news.find('span', {'class': 'news__kicker'}).text.encode('latin').decode()
    title = news.find('span', {'class': 'news__title'}).text.encode('latin').decode()
    link = news.find('a', {'class': 'news__card'}).attrs['href']
    print(date)
    print(source)
    print(kicker)
    print(title)
    print(link)


11.08.22
dpa-afx
Optimismus
Deutsche Telekom-Aktie schließt freundlich: Deutsche Telekom erhöht erneut das Gewinnziel - Cashflow höher
/nachricht/aktien/optimismus-deutsche-telekom-aktie-schliesst-freundlich-deutsche-telekom-erhoeht-erneut-das-gewinnziel-cashflow-hoeher-11620885
04.08.22
dpa-afx
Vertragszahlen gestiegen
United Internet-Aktie dreht ins Plus: United Internet punktet bei Kundenverträgen - Neues Regelwerk belastet 1&1
/nachricht/aktien/vertragszahlen-gestiegen-united-internet-aktie-dreht-ins-plus-united-internet-punktet-bei-kundenvertraegen-neues-regelwerk-belastet-1-1-11597372
03.08.22
finanzen.net
Analysen im Überblick
Ausblick: 1&1 informiert über die jüngsten Quartalsergebnisse
/nachricht/aktien/analysen-im-ueberblick-ausblick-1-1-informiert-ueber-die-juengsten-quartalsergebnisse-11592163
31.07.22
finanzen.net
Analysteneinschätzungen
Juli 2022: So schätzen Experten die 1&1-Aktie ein
/nachricht/aktien/juli-2022-so-schaetzen-experten-die-1-1-aktie-ein-11583037
27.07.22
d

In [124]:
link.attrs['href']

'/nachricht/aktien/analysten-vor-bilanz-ausblick-1-1-oeffnet-die-buecher-zum-abgelaufenen-quartal-10716781'